VARSAYIMLAR
-------------
1 bağımsız değişken, 2 bağımlı değişken

Gözlemler bağımsız

Her bağımlı değişken sürekli

Her bağımlı değişken normal dağılmalı (ya da n > 20)

Varyanslar homojen

Varyans-kovaryans matrisi eşitliği


#### Farklı ürün gruplarının kadın ve erkek gruplarının memnuniyet puanına etkisi ?

In [8]:
import pandas as pd
data = pd.read_excel("C:/Users/htcso/OneDrive/Masaüstü/pythonProject/datasets/one_way_manova_ornek.xlsx")
data.head()

,Ürün,ErkekTutum,KadınTutum
0,Ürün 1,10,6
1,Ürün 1,5,1
2,Ürün 1,6,6
3,Ürün 1,3,10
4,Ürün 1,7,8


In [2]:
# Her ürün grubundan 20 adet örneklem old. için verilerin normal dağıldığı varsayılıyor.

##### Homojenlik Testi

In [9]:
import pingouin as pg
homojenlik = pg.homoscedasticity(data, dv="ErkekTutum", group="Ürün", center="mean")
print(homojenlik)

               W      pval  equal_var
levene  1.672707  0.196821       True


In [4]:
# pval 0.196821, equal_var True --> homojenlik varsayımı geçerlidir.

In [10]:
homojenlik = pg.homoscedasticity(data, dv="KadınTutum", group="Ürün", center="mean")
print(homojenlik)

               W      pval  equal_var
levene  0.900525  0.412058       True


In [11]:
# pval 0.412058, equal_var True --> homojenlik varsayımı geçerlidir.

##### Varyans - Kovaryans Matrisi Eşitliği

In [12]:
varcov = pg.box_m(data, dvs=["ErkekTutum", "KadınTutum"], group="Ürün")
print(varcov)

         Chi2   df      pval  equal_cov
box  3.018976  6.0  0.806461       True


In [13]:
# pval 0.806461, equal_cov True --> var-kov eşitliği varsayımı geçerlidir.

##### Manova

In [14]:
from statsmodels.multivariate.manova import MANOVA
model = MANOVA.from_formula("ErkekTutum + KadınTutum ~ Ürün", data=data)
print(model.mv_test())

                  Multivariate linear model
                                                             
-------------------------------------------------------------
        Intercept        Value  Num DF  Den DF F Value Pr > F
-------------------------------------------------------------
           Wilks' lambda 0.3065 2.0000 56.0000 63.3568 0.0000
          Pillai's trace 0.6935 2.0000 56.0000 63.3568 0.0000
  Hotelling-Lawley trace 2.2627 2.0000 56.0000 63.3568 0.0000
     Roy's greatest root 2.2627 2.0000 56.0000 63.3568 0.0000
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
          Ürün          Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.9099 4.0000 112.0000  1.3532 0.2548
         Pillai's trace 0.0907 4.0000 114.0000  1.3540 0.2544
 Hotelling-Lawley trace 0.

In [15]:
# Wilks' lambda varsayımlara karşı daha güçlü olduğu için, daha çok kullanılır.
# Pr > F: 0.2548 --> Ürün gruplarının Kadın Erkek gruplarının tutumları üzerinden anlamlı bir etkisi yoktur denebilir.

##### Test Anlamlı Çıksaydı Post Hoc Testi

In [16]:
ph1 = pg.pairwise_tukey(data, dv="ErkekTutum", between="Ürün")
ph2 = pg.pairwise_tukey(data, dv="KadınTutum", between="Ürün")
print(ph1)
print(ph2)

        A       B  mean(A)  mean(B)  diff        se         T   p-tukey  \
0  Ürün 1  Ürün 2     6.55     4.85   1.7  0.964683  1.762236  0.191525   
1  Ürün 1  Ürün 3     6.55     5.05   1.5  0.964683  1.554914  0.273484   
2  Ürün 2  Ürün 3     4.85     5.05  -0.2  0.964683 -0.207322  0.976593   

     hedges  
0  0.571069  
1  0.495975  
2 -0.060157  
        A       B  mean(A)  mean(B)  diff        se         T   p-tukey  \
0  Ürün 1  Ürün 2     4.85     5.35 -0.50  0.965274 -0.517988  0.862897   
1  Ürün 1  Ürün 3     4.85     6.00 -1.15  0.965274 -1.191371  0.463139   
2  Ürün 2  Ürün 3     5.35     6.00 -0.65  0.965274 -0.673384  0.779841   

     hedges  
0 -0.161225  
1 -0.356179  
2 -0.216009  


In [17]:
# p-tukey < 0.05 olan gruplar anlamlı fark yaratmaktadır. Manova testimiz zaten fark yoktur sonucunu ortaya çıkardığı için
# post hoc testinde de gruplar arası fark görülmemektedir